In [1]:
import pandas as pd

In [2]:
data=pd.read_csv('/content/predicted_values.csv')

In [3]:
## importing packages
import numpy as np
import pandas as pd

In [4]:
df = pd.read_csv("/content/predicted_values.csv", parse_dates = ["date"])
#df["date"] = df.Datetime.dt.date.astype(str)

<ipython-input-4-67d04305bab2>:1: UserWarning: Parsing dates in DD/MM/YYYY format when dayfirst=False (the default) was specified. This may lead to inconsistently parsed dates! Specify a format to ensure consistent parsing.
  df = pd.read_csv("/content/predicted_values.csv", parse_dates = ["date"])


In [5]:
## PM2.5 Sub-Index calculation
def get_PM25_subindex(x):
    if x <= 30:
      return x * 50 / 30
    elif x <= 60:
      return 50 + (x - 30) * 50 / 30
    elif x <= 90:
      return 100 +(x - 60) * 100 / 30
    elif x <= 120:
      return 200 +(x - 90) * 100 / 30
    elif x <= 250:
      return 300 +(x - 120) * 100 / 130
    elif x > 250:
        return 400 +(x - 250) * 100 / 130
    else:
        return 0
df["PM2.5_SubIndex"] = df["pm2_5"].apply(lambda x: get_PM25_subindex(x))

In [6]:
## CO Sub-Index calculation
def get_CO_subindex(x):
    if x <= 1:
      return x * 50 / 1
    elif x <= 2:
      return 50 + (x - 1) * 50 / 1
    elif x <= 10:
      return 100 +(x - 2) * 100 / 8
    elif x <= 17:
      return 200 +(x - 10) * 100 / 7
    elif x <= 34:
      return 300 +(x - 17) * 100 / 17
    elif x > 34:
      return 400 +(x - 34) * 100 / 17
    else:
      return 0

df["CO_SubIndex"] = df["co"].apply(lambda x: get_CO_subindex(x))

In [7]:
## O3 Sub-Index calculation
def get_O3_subindex(x):
    if x <= 50:
      return x * 50 / 50
    elif x <= 100:
      return 50 + (x - 50) * 50 / 50
    elif x <= 168:
      return 100 +(x - 100) * 100 / 68
    elif x <= 208:
      return 200 +(x - 168) * 100 / 40
    elif x <= 748:
      return 300 +(x - 208) * 100 / 539
    elif x > 748:
      return 400 +(x - 400) * 100 / 539
    else:
      return 0

df["O3_SubIndex"] = df["o3"].apply(lambda x: get_O3_subindex(x))

In [8]:
## NO2 Sub-Index calculation
def get_NO2_subindex(x):
    if x <= 40:
      return x * 50 / 40
    elif x <= 80:
      return 50 + (x - 40) * 50 / 40
    elif x <= 180:
      return 100 +(x - 80) * 100 / 100
    elif x <= 280:
      return 200 +(x - 180) * 100 / 100
    elif x <= 400:
      return 300 +(x - 280) * 100 / 120
    elif x > 400:
      return 400 +(x - 400) * 100 / 120
    else:
      return 0

df["NO2_SubIndex"] = df["no2"].apply(lambda x: get_NO2_subindex(x))

In [9]:
## SO2 Sub-Index calculation
def get_SO2_subindex(x):
    if x <= 40:
        return x * 50 / 40
    elif x <= 80:
        return 50 + (x - 40) * 50 / 40
    elif x <= 380:
        return 100 + (x - 80) * 100 / 300
    elif x <= 800:
        return 200 + (x - 380) * 100 / 420
    elif x <= 1600:
        return 300 + (x - 800) * 100 / 800
    elif x > 1600:
        return 400 + (x - 1600) * 100 / 800
    else:
        return 0

df["SO2_SubIndex"] = df["so2"].apply(lambda x: get_SO2_subindex(x))

In [10]:
## AQI bucketing
def get_AQI_bucket(x):
    if x <= 50:
        return "Good"
    elif x <= 100:
        return "Satisfactory"
    elif x <= 200:
        return "Moderate"
    elif x <= 300:
        return "Poor"
    elif x <= 400:
        return "Very Poor"
    elif x > 400:
        return "Severe"
    else:
        return np.NaN

In [11]:
df["Checks"] = (df["PM2.5_SubIndex"] > 0).astype(int) + \
               (df["SO2_SubIndex"] > 0).astype(int) + \
               (df["NO2_SubIndex"] > 0).astype(int) + \
               (df["CO_SubIndex"] > 0).astype(int) + \
               (df["O3_SubIndex"] > 0).astype(int)

df["AQI_calculated"] = round(df[["PM2.5_SubIndex", "SO2_SubIndex", "NO2_SubIndex", "CO_SubIndex", "O3_SubIndex"]].max(axis = 1))
df.loc[df["PM2.5_SubIndex"]<= 0, "AQI_calculated"] = np.NaN
df.loc[df.Checks < 3, "AQI_calculated"] = np.NaN

df["AQI_bucket_calculated"] = df["AQI_calculated"].apply(lambda x: get_AQI_bucket(x))
df[~df.AQI_calculated.isna()].head(5)

,date,no2,o3,so2,pm2_5,co,PM2.5_SubIndex,CO_SubIndex,O3_SubIndex,NO2_SubIndex,SO2_SubIndex,Checks,AQI_calculated,AQI_bucket_calculated
0,2022-10-10,30.50,0.27,8.34,20.94,0.125116,34.900000,6.255813,0.27,38.1250,10.4250,5,38.0,Good
1,2022-11-10,28.79,0.01,9.42,24.19,0.209783,40.316667,10.489158,0.01,35.9875,11.7750,5,40.0,Good
2,2022-12-10,41.47,0.46,9.54,60.88,0.232361,102.933333,11.618059,0.46,51.8375,11.9250,5,103.0,Moderate
3,2022-10-13,60.32,0.00,13.95,68.54,0.601129,128.466667,30.056452,0.00,75.4000,17.4375,4,128.0,Moderate
4,2022-10-14,47.98,0.22,17.40,34.15,0.239886,56.916667,11.994312,0.22,59.9750,21.7500,5,60.0,Satisfactory


In [12]:
df.to_csv('AQI.csv', index=False)

In [13]:
# Aggregate AQI values for each pollutant
aqi_sum = df[["PM2.5_SubIndex", "SO2_SubIndex", "NO2_SubIndex", "CO_SubIndex", "O3_SubIndex"]].sum()

# Find the pollutant contributing the most to AQI
max_contributor = aqi_sum.idxmax()
max_aqi_contribution = aqi_sum[max_contributor]
print(max_contributor)

PM2.5_SubIndex


In [32]:
#df = pd.read_csv('/content/AQI.csv',encoding='utf-8')
# Filter rows based on values in the 'AQI_bucket_calculated' column
# Display the filtered DataFrame
Poor = df[df['AQI_bucket_calculated'].isin(['Poor'])]
Poor.head(11)


,date,no2,o3,so2,pm2_5,co,PM2.5_SubIndex,CO_SubIndex,O3_SubIndex,NO2_SubIndex,SO2_SubIndex,Checks,AQI_calculated,AQI_bucket_calculated
13,2022-10-23,16.28,64.37,3.55,96.31,0.157102,221.033333,7.855102,64.370000,20.3500,4.4375,5,221.0,Poor
16,2022-10-26,56.21,1.56,7.03,91.12,0.247411,203.733333,12.370566,1.560000,70.2625,8.7875,5,204.0,Poor
38,2022-11-17,47.30,2.46,6.32,95.64,0.213546,218.800000,10.677284,2.460000,59.1250,7.9000,5,219.0,Poor
39,2022-11-18,43.87,29.68,7.57,91.08,0.168390,203.600000,8.419482,29.680000,54.8375,9.4625,5,204.0,Poor
42,2022-11-21,27.08,39.70,4.05,108.66,0.093130,262.200000,4.656524,39.700000,33.8500,5.0625,5,262.0,Poor
58,2022-07-12,33.93,21.82,5.36,93.19,0.153339,210.633333,7.666975,21.820000,42.4125,6.7000,5,211.0,Poor
129,2023-02-17,16.62,128.75,20.50,101.95,0.070552,239.833333,3.527623,142.279412,20.7750,25.6250,5,240.0,Poor
135,2023-02-23,13.02,153.06,21.46,97.45,0.063027,224.833333,3.151369,178.029412,16.2750,26.8250,5,225.0,Poor
136,2023-02-25,24.68,83.68,21.46,100.82,0.125116,236.066667,6.255813,83.680000,30.8500,26.8250,5,236.0,Poor
217,2023-05-17,85.00,0.00,21.93,106.38,1.000000,254.600000,50.000000,0.000000,105.0000,27.4125,4,255.0,Poor


In [15]:
aqi_sum = Poor[["PM2.5_SubIndex", "SO2_SubIndex", "NO2_SubIndex", "CO_SubIndex", "O3_SubIndex"]].sum()
max_contributor = aqi_sum.idxmax()
max_aqi_contribution = aqi_sum[max_contributor]
print(max_contributor)

PM2.5_SubIndex


In [16]:
Very_Poor = df[df['AQI_bucket_calculated'].isin(['Very Poor'])]
Very_Poor.head()

,date,no2,o3,so2,pm2_5,co,PM2.5_SubIndex,CO_SubIndex,O3_SubIndex,NO2_SubIndex,SO2_SubIndex,Checks,AQI_calculated,AQI_bucket_calculated
22,2022-01-11,27.42,50.07,6.50,132.56,0.104421,309.661538,5.221045,50.07,34.2750,8.1250,5,310.0,Very Poor
40,2022-11-19,42.16,33.62,6.32,132.35,0.172152,309.500000,8.607608,33.62,52.7000,7.9000,5,310.0,Very Poor
41,2022-11-20,45.24,23.96,5.72,145.74,0.198493,319.800000,9.924637,23.96,56.5500,7.1500,5,320.0,Very Poor
59,2022-08-12,45.93,0.00,7.33,163.05,0.458136,333.115385,22.906791,0.00,57.4125,9.1625,4,333.0,Very Poor
74,2022-12-24,22.28,64.37,4.41,147.20,0.096893,320.923077,4.844651,64.37,27.8500,5.5125,5,321.0,Very Poor


In [17]:
aqi_sum = Very_Poor[["PM2.5_SubIndex", "SO2_SubIndex", "NO2_SubIndex", "CO_SubIndex", "O3_SubIndex"]].sum()
max_contributor = aqi_sum.idxmax()
max_aqi_contribution = aqi_sum[max_contributor]
print(max_contributor)

PM2.5_SubIndex


In [19]:
Severe = df[df['AQI_bucket_calculated'].isin(['Severe'])]
Severe.head()

,date,no2,o3,so2,pm2_5,co,PM2.5_SubIndex,CO_SubIndex,O3_SubIndex,NO2_SubIndex,SO2_SubIndex,Checks,AQI_calculated,AQI_bucket_calculated


In [24]:
Good = df[df['AQI_bucket_calculated'].isin(['Good'])]
Good.head()

,date,no2,o3,so2,pm2_5,co,PM2.5_SubIndex,CO_SubIndex,O3_SubIndex,NO2_SubIndex,SO2_SubIndex,Checks,AQI_calculated,AQI_bucket_calculated
0,2022-10-10,30.50,0.27,8.34,20.94,0.125116,34.900000,6.255813,0.27,38.1250,10.4250,5,38.0,Good
1,2022-11-10,28.79,0.01,9.42,24.19,0.209783,40.316667,10.489158,0.01,35.9875,11.7750,5,40.0,Good
5,2022-10-15,35.99,2.30,7.87,23.43,0.172152,39.050000,8.607608,2.30,44.9875,9.8375,5,45.0,Good
6,2022-10-16,31.53,6.35,10.49,18.00,0.100658,30.000000,5.032919,6.35,39.4125,13.1125,5,39.0,Good
7,2022-10-17,24.33,13.77,6.62,19.68,0.093130,32.800000,4.656524,13.77,30.4125,8.2750,5,33.0,Good


In [22]:
Satisfactory = df[df['AQI_bucket_calculated'].isin(['Satisfactory'])]
Satisfactory.head()

,date,no2,o3,so2,pm2_5,co,PM2.5_SubIndex,CO_SubIndex,O3_SubIndex,NO2_SubIndex,SO2_SubIndex,Checks,AQI_calculated,AQI_bucket_calculated
4,2022-10-14,47.98,0.22,17.40,34.15,0.239886,56.916667,11.994312,0.22,59.9750,21.7500,5,60.0,Satisfactory
9,2022-10-19,33.93,0.00,10.85,43.97,0.420508,73.283333,21.025382,0.00,42.4125,13.5625,4,73.0,Satisfactory
10,2022-10-20,37.70,0.00,11.56,32.74,0.330196,54.566667,16.509777,0.00,47.1250,14.4500,4,55.0,Satisfactory
15,2022-10-25,19.36,82.97,10.85,57.94,0.047977,96.566667,2.398862,82.97,24.2000,13.5625,5,97.0,Satisfactory
18,2022-10-28,24.33,32.19,5.13,45.80,0.079960,76.333333,3.998010,32.19,30.4125,6.4125,5,76.0,Satisfactory


In [23]:
Moderate = df[df['AQI_bucket_calculated'].isin(['Moderate'])]
Moderate.head()

,date,no2,o3,so2,pm2_5,co,PM2.5_SubIndex,CO_SubIndex,O3_SubIndex,NO2_SubIndex,SO2_SubIndex,Checks,AQI_calculated,AQI_bucket_calculated
2,2022-12-10,41.47,0.46,9.54,60.88,0.232361,102.933333,11.618059,0.46,51.8375,11.9250,5,103.0,Moderate
3,2022-10-13,60.32,0.00,13.95,68.54,0.601129,128.466667,30.056452,0.00,75.4000,17.4375,4,128.0,Moderate
8,2022-10-18,43.18,0.00,11.92,61.93,0.435558,106.433333,21.777889,0.00,53.9750,14.9000,4,106.0,Moderate
14,2022-10-24,18.16,41.84,4.77,79.23,0.074318,164.100000,3.715891,41.84,22.7000,5.9625,5,164.0,Moderate
17,2022-10-27,39.76,21.10,6.50,65.95,0.142049,119.833333,7.102454,21.10,49.7000,8.1250,5,120.0,Moderate


In [25]:
len(Good)

57

In [26]:
len(Moderate)

43

In [27]:
len(Satisfactory)

110

In [28]:
len(Poor)

11

In [30]:
len(Very_Poor)

5

In [31]:
len(Severe)

0